In [ ]:
from __future__ import print_function
%matplotlib inline
import matplotlib.pylab as plt
import sys, os, glob
import numpy as np

plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

import findspark, os
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext

# put the number of executors and cores into variables so we can refer to it later
num_execs = 200
exec_cores = 4

# initializing the SparkConf
os.environ['SPARK_DRIVER_MEMORY'] = '4g'
os.environ['SPARK_CONF_DIR'] = '%s/spark_config'%os.getcwd()
conf = (SparkConf()
            .set('spark.executor.memory', '8g')
            .set('spark.executor.instances', str(num_execs))
            .set('spark.executor.cores', str(exec_cores))
            .set('spark.storage.memoryFraction', 0.6)
            .set('spark.shuffle.memoryFraction', 0.2)
            .set('spark.yarn.executor.memoryOverhead', 2048)
            .set('spark.executor.extraJavaOptions', '-XX:+UseCompressedOops')
            .set('spark.yarn.am.memory', '8g')
            .set('spark.yarn.am.cores', 12)
            .set('spark.driver.maxResultSize', '4096')
            .set('spark.akka.frameSize', 256)
            .set('spark.akka.threads', 6)
            .set('spark.executorEnv.PYTHONPATH', 
                 '/cluster/apps/spark/spark-current/python:/cluster/apps/spark/spark-current/python/lib/py4j-0.8.2.1-src.zip')
            .set('spark.executorEnv.PATH', os.environ['PATH']))



In [ ]:
sc = SparkContext(master = 'yarn-client', conf = conf)

In [ ]:
from pyspark.sql import SQLContext

In [ ]:
sqc = SQLContext(sc)

In [ ]:
%%time
data = sqc.read.parquet('/user/roskarr/twitter/2014_*')

In [ ]:
data.count()

In [ ]:
data2.groupBy(data2.retweet_count).count().show()

In [ ]:
data2.groupBy(data2.retweeted).count().show()

In [ ]:
data2.printSchema()

In [ ]:
data_filt = data2.where(data2.retweeted == 'True')
data_filt.count()

In [ ]:
data2.take(5)a

In [ ]:
from snakebite import client

cl = client.Client('hadoop.ethz.ch', 8020)

import re

for d in filter(lambda x: x['file_type'] == 'd', cl.ls(['/user/imoise/TwitterData'])) :
    year_month = re.findall('/(\w+_\w+)',d['path'])
    if len(year_month) > 0 : 
        year_month = year_month[0]
        (sqc.read.json('%s/*/*'%d['path'])
                 .coalesce(500)
                 .write.mode('overwrite')
                 .parquet('/user/roskarr/twitter/%s'%year_month))

In [ ]:
### sc.stop()